In [13]:
import random
from datetime import datetime

In [14]:
import boto3
from botocore.exceptions import ClientError
import psycopg2
import json
from pprint import pprint

In [15]:
def get_secret():

    secret_name = "rds!db-*************************"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    secret = get_secret_value_response['SecretString']
    return secret

In [16]:

password = json.loads(get_secret())
connection = psycopg2.connect( user="postgres", password=password['password'], host="database-1.*************.rds.amazonaws.com", port=5432)

cursor = connection.cursor()

In [5]:
pages_table = """
create table if not exists pages (
page_id int not null primary key,
page_name varchar(20))
"""


page_likes = """
create table if not exists page_likes (
user_id int not null,
page_id int not null
)"""

index_query = """ CREATE INDEX page_likes_page_id ON page_likes (page_id); """



In [6]:
solution = """SELECT count(*)
from (
select page_id
FROM pages
EXCEPT
SELECT page_id
FROM page_likes); """

start_time = datetime.now()
cursor.execute(solution)
for i in cursor.fetchall():
    print(i)
end_time = datetime.now()
print(end_time-start_time)

(183934974,)
0:14:12.606339


In [7]:
solution = """select count(*) from (
SELECT pages.page_id
FROM pages left join page_likes
on pages.page_id = page_likes.page_id
where page_likes.page_id is null)

; """


start_time = datetime.now()
cursor.execute(solution)
for i in cursor.fetchall():
    print(i)
end_time = datetime.now()
print(end_time-start_time)

(183934974,)
0:03:44.442546


In [8]:
solution = """select count(*) from 
(SELECT pages.page_id
FROM pages 
where not exists (select 1 from page_likes
                           where page_likes.page_id = pages.page_id)
                           )
; """


start_time = datetime.now()
cursor.execute(solution)
for i in cursor.fetchall():
    print(i)
end_time = datetime.now()
print(end_time-start_time)

(183934974,)
0:03:45.634451


In [9]:
solution = """select count(*) from (
SELECT pages.page_id
FROM pages 
where not exists (select 1 from page_likes
                           where page_likes.page_id = pages.page_id
                           limit 1))
; """

start_time = datetime.now()
cursor.execute(solution)
for i in cursor.fetchall():
    print(i)
end_time = datetime.now()
print(end_time-start_time)

(183934974,)
0:03:45.440492


In [18]:
solution = """select count(*) from (SELECT page_id
FROM pages
where page_id not in (SELECT page_id
FROM page_likes))
; """


start_time = datetime.now()
cursor.execute(solution)
for i in cursor.fetchall():
    print(i)
end_time = datetime.now()
print(end_time-start_time)

OperationalError: SSL connection has been closed unexpectedly


In [12]:
connection.close()